In [ ]:
import pandas as pd
import sys
import os
import nltk
import csv
import string
import numpy as np
import re
from dateutil.parser import parse
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
header_file_desc = pd.read_csv('header_information.csv')
header_file_desc.head()

In [ ]:
class preProcess:
    def __init__(self):
        pass
    def text_process(self, text):
        '''
        Takes in a string of text, then performs the following:
        1. Remove all punctuation
        2. Remove all stopwords
        3. Return the cleaned text as a list of words
        4. Remove words
        '''
        stemmer = WordNetLemmatizer()
        nopunc = [char for char in text if char not in string.punctuation]
        nopunc = ''.join([i for i in nopunc if not i.isdigit()])
        nopunc =  [word for word in nopunc.split() if word not in stopwords.words('english')]
        return [stemmer.lemmatize(word) for word in nopunc]
    

    def pre_process(self, data_row):
        preprocess_data_row = []
        for dataelem in data_row:
            preprocess_data_row.append(self.text_process(dataelem))
        return preprocess_data_row
        
    

In [ ]:
class Features:
    def __init__(self):
        self.preprocess = preProcess()
    def isNumeric(self, char):
        import re
        regex = re.findall(r"^[-+]?(?:\d*\.\d+|\d+$)", char)
        if regex:
            return True
        for c in char:
            if not c.isdigit():
                return False
        return True
    def isOnlyAlpha(self, char):
        regex = re.findall(r"[A-Za-z]+$", char)
        if regex:
            return True
        return False
    
    def isBlank(self, each_element):
        if not each_element or each_element == 'UNKNOWN' or each_element.isspace() or each_element == 'NULL' or each_element == ' ' or each_element == '' or each_element == None or each_element == "" or each_element ==" ":
            return True
        return False
    
    def specialCharacters(self,char):
        import re
        regex = re.findall(r'[\w]',char)
        if regex:
            return True
        return False
    

    def isDate(self, string, fuzzy=False):
        """
        Return whether the string can be interpreted as a date.

        :param string: str, string to check for date
        :param fuzzy: bool, ignore unknown tokens in string if True
        """
        try: 
            parse(string, fuzzy=fuzzy)
            return True

        except ValueError:
            return False

    def noSpecialCharacters(self, char):
        noSpecialChars = 0
        noSpecialChars =  len(char) - len( re.findall('[\w]', char) ) + len(re.findall('[-_]+',char))
        return noSpecialChars
    
    def tokenize( self, string, separator = ',', quote = '"' ):
        """
        Split a comma separated string into a List of strings.

        Separator characters inside the quotes are ignored.

        :param string: A string to be split into chunks
        :param separator: A separator character
        :param quote: A character to define beginning and end of the quoted string
        :return: A list of strings, one element for every chunk
        """
        comma_separated_list = []

        chunk = ''
        in_quotes = False

        for character in string:
            if character == separator and not in_quotes:
                comma_separated_list.append(chunk)
                chunk = ''

            else:
                chunk += character
                if character == quote:
                    in_quotes = False if in_quotes else True
        # print(chunk)
        # print(chunk.replace('"',''))
        
        comma_separated_list.append(chunk.replace('"',''))
        return comma_separated_list
    
    def featureList(self, file_name):
        file = open(file_name, 'r')
        lines = file.readlines()
        feature_list = []
        feature_list_vector = []
        for line in lines:
            feature = {}
            # each_row_elements = [ '{}'.format(x.replace("",'')) for x in list(csv.reader([line], delimiter=',', quotechar='"'))[0] ]
            # each_row_elements = line.split(r",(?=(?:[^\"']*[\"'][^\"']*[\"'])*[^\"']*$)")
            each_row_elements = self.tokenize(line)
            num_numerics = 0
            num_alphs = 0
            num_null = 0
            # print(each_row_elements)
            for each_element in each_row_elements:
                
                
                if self.isBlank(each_element):
                    num_null = num_null + 1
                elif not self.isNumeric(each_element):
                    num_alphs = num_alphs + 1
                    # print('alphs',each_element)
                    each_element = self.preprocess.text_process(each_element)
                else:
                    # print('numeric',each_element)
                    num_numerics = num_numerics + 1
#             if num_null == 0:
#                 if num_numerics == 0:
                    
#                     print('header line')
#                     print(line)
#             else :
#                 pass
            feature['num_numerics'] = num_numerics
            feature['num_null'] = num_null
            feature['num_alphs'] = num_alphs
            feature_list.append(feature)
            if num_numerics == 0 and num_null == 0 and num_alphs == 0:
                print("tha values seem to be empty")
            else:
                feature_list_vector.append([num_numerics, num_null, num_alphs])
        df_feature_list = pd.DataFrame(columns = ['num_numerics', 'num_null', 'num_alphs'])
        df_feature_list['num_numerics'] = [feature[0] for feature in feature_list_vector]
        df_feature_list['num_null'] = [feature[1] for feature in feature_list_vector]
        df_feature_list['num_alphs'] = [feature[2] for feature in feature_list_vector]
        return lines,feature_list, feature_list_vector, df_feature_list 
    
    
    def rule_based_classification(self, file):
        lines, feature_list, feature_list_vector, df_feature_list = self.featureList(file)
        
        #rule 2 -> check for the no null values in the header file and no numerics
    
        idx = df_feature_list.index[(df_feature_list['num_numerics'] == 0) & (df_feature_list['num_null'] == 0)].to_list()
       
        #rule 3 -> incase more than one elements in the list is given then apply the classification algorithm for the same
        print(idx)
        if len(idx) > 1:
            
            pred_indices = []
            
            # each_row_elements = [ '{}'.format(x.replace("",'')) for x in list(csv.reader([lines[idx[0]]], delimiter=',', quotechar='"'))[0] ]
            each_row_elements = self.tokenize(lines[idx[0]])
            column_len = len(each_row_elements)
            # print("column len", column_len)
            
            data = []
            for i,index in enumerate(idx):
                
                data_row = lines[index]
                # print(data_row)
                # data_row = [ '{}'.format(x.replace("",'')) for x in list(csv.reader([data_row], delimiter=',', quotechar='"'))[0] ]
                # print(data_row)
                data_row = self.tokenize(data_row)
                data.append(data_row)
                print(data_row)
                # print(data_row)
                # df_text.loc[i] = data_row
                # df_sub_feature = pd.DataFrame(columns = ['num_special_chars', 'data_len', 'bool_numeric', 'bool_alpha'])
                # for data in data_row:
                #     num_special_chars = self.noSpecialCharacters(data)
                #     data_len = len(data)
                #     bool_numeric = 1 if self.isNumeric(data) else 0
                #     bool_alpha = 1 if self.isOnlyAlpha(data) else 0
                #     sub_feature_vector.append([num_special_chars, data_len, bool_numeric, bool_alpha])
                # print(sub_feature_vector)
                # kmeans.fit(sub_feature_vector)
                # identified_clusters = kmeans.fit_predict(sub_feature_vector)
                # print(identified_clusters)
            df_text = pd.DataFrame(data, columns = [ x for x in range(column_len)])
            # print(df_text)
            for i in range(column_len):
                sub_feature_vector = []
                data_row = df_text[i]
                print(data_row)
                for data in data_row:
                    if data is not None:
                        data = data.strip('\"')
                        num_special_chars = self.noSpecialCharacters(data)
                        data_len = len(data)
                        bool_numeric = 1 if self.isNumeric(data) else 0
                        bool_alpha = 1 if self.isOnlyAlpha(data) else 0
                        bool_date = 1 if self.isDate(data) else 0
                        sub_feature_vector.append([num_special_chars, data_len, bool_numeric, bool_alpha, bool_date])
                    
                print(sub_feature_vector)
                scaler = StandardScaler()
                sub_feature_vector = scaler.fit_transform(sub_feature_vector)
                
                kmeans.fit(sub_feature_vector)
                identified_clusters = kmeans.fit_predict(sub_feature_vector)
                print(identified_clusters)
                map_index = {}
                
                count_arr = np.bincount(identified_clusters)
                if len(count_arr) == 1:
                    count_arr = np.append(count_arr, [0])
                if count_arr[0] > 1 and count_arr[1]>1:
                    pass
                elif count_arr[0] == 1 and count_arr[1] > 1:
                    for i in range(len(identified_clusters)):
                        if identified_clusters[i]==0:
                            if i in map_index.keys():
                                map_index[i] = map_index[i] + 1
                            else:
                                map_index[i] = 0
                elif count_arr[0] > 1 and count_arr[1] == 1:
                    for i in range(len(identified_clusters)):
                        if identified_clusters[i]== 1:
                            if i in map_index.keys():
                                map_index[i] = map_index[i] + 1
                            else:
                                map_index[i] = 0
                else:
                    pass
                max_occ = -1
                max_index = 0
                for i in range(len(identified_clusters)):
                    
                    if i in map_index.keys() and map_index[i] > max_occ :
                        max_occ = map_index[i]
                        max_index = i
                    
                
                print("max_index",max_index)
                value = self.isHeader(
                
                
        def isHeader(self, df):
            n=5, th=0.9
            # df1 = pd.read_csv(path, header='infer', nrows=n)
            # df2 = pd.read_csv(path, header=None, nrows=n)
            df1 = pd.DataFrame(data = df.values, header = 'infer')
            df2 = pd.DataFrame(data = df.values, header = None)
            sim = (df1.dtypes.values == df2.dtypes.values).mean()
            return value = 'infer' if sim < th else None
            
                    
                    
                    
                    

        
        


In [ ]:
help(pyparsing)

In [ ]:
[ x for x in range(3)]

In [ ]:
help(pyparsing)

In [ ]:
arr = np.array([0,0])
count_arr = np.bincount(arr)
print(count_arr)

In [ ]:
class oneSVMModel:
    def __init__(self):
        pass
    def getModel(self):
        clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
        return clf
model = oneSVMModel().getModel()

        

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(2,max_iter=400, algorithm = 'auto')


In [ ]:
csv_file_name = 'header_information.csv'
df = pd.read_csv(csv_file_name)
file_names = df['file_name']
headers = df['has_header']
df_header = pd.DataFrame(columns = ['file_name','num_numerics','num_null','num_alphs','header_present'])
for file_name, header in zip(file_names, headers):
    #rule 1
    if header == 'yes':
        file = 'data/'+file_name
        print(file)
        Features().rule_based_classification(file)
        #rules
        #rule 2-> if none of the elements in the row is NULL and not just numerics
        # selected_idx_rows = 
        # print(df_feature_list)
        # print(file)
        # model.fit(feature_list_vector)
        # y_pred_train = model.predict(feature_list_vector)
        # print('y_pred_train', y_pred_train)
        # kmeans.fit(y_pred_train.reshape(-1,1))
        # identified_clusters = kmeans.fit_predict(y_pred_train.reshape(-1,1))
        # print('cluster', identified_clusters)
        
    else:
        print('no header present')

In [ ]:
file_name = 'data/0ee50c27-22c9-4323-b41e-3f9f71257ba1.txt'
Features().rule_based_classification(file_name)


In [313]:
def print_headers(csv_file_name):
    df = pd.read_csv(csv_file_name)
    file_names = df['file_name']
    headers = df['has_header']
    for file_name, header in zip(file_names, headers):
        if header == 'yes':
            file = open('data/'+file_name, 'rb')
            lines = file.readlines()
            print(lines[0])
        else:
            print('no header present')

In [ ]:
print_headers('header_information.csv')